In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ipywidgets import Output, HBox, VBox
from IPython import display
import pandas as pd
import ipyanchorviz as av

In [3]:
av.__version__

'0.1.0'

In [4]:
viz = av.AnchorViz(
    anchors=[
        {"id": "0", "name": "test anchor", "color": "red", "theta": -0.5},
        {"id": "1", "name": "test anchor 2", "theta": -0.9}
    ],
    data = [
        {"id": "0", "color": "orange", "weights": { "0": 0.5, "1": 0.5}},
        {"id": "1", "color": "orange", "weights": { "0": 0.0, "1": 0.0}},
        {"id": "2", "color": "orange", "weights": { "0": 1.0, "1": 1.1}},
        {"id": "3", "color": "blue", "weights": { "0": 1.0, "1": 0.0}, "labeled": True},
        {"id": "4", "color": "blue", "weights": { "0": 1.0, "1": 0.5}},
        {"id": "5", "color": "orange", "weights": { "0": 0.0, "1": 1.0}, "labeled": True},
        {"id": "6", "color": "blue", "weights": { "0": 0.0, "1": 0.5}},
    ],
    autoNorm = True
)

In [5]:
import logging

def new_anchor_added(content):
    logging.info("new anchor!")
def anchor_dragged(content):
    logging.info("anchor dragged")

viz.on_anchor_add(new_anchor_added)
viz.on_anchor_drag(anchor_dragged)

In [6]:
anchor_display = Output()
data_display = Output()
lassoed_data_display = Output()
selected_anchor_display = Output()
selected_point_display = Output()
what_display = Output()
def anchorchanges(change):
    with anchor_display:
        display.clear_output(wait=True)
        display.display(pd.DataFrame(viz.anchors))
def datachanges(change):
    with data_display:
        display.clear_output(wait=True)
        display.display(pd.DataFrame(viz.data))
def lassochanges(change):
    data = pd.DataFrame(viz.data)
    with lassoed_data_display:
        display.clear_output(wait=True)
        if "id" in data.columns:
            display.display(data[data.id.isin(viz.lassoedPointIDs)])
def selectedanchorchanges(change):
    with selected_anchor_display:
        display.clear_output(wait=True)
        print("selected anchor ID:", viz.selectedAnchorID)
def selecteddatachanges(change):
    with selected_point_display:
        display.clear_output(wait=True)
        print("selected data point ID:", viz.selectedDataPointID)
viz.observe(anchorchanges, names="anchors")
viz.observe(datachanges, names="data")
viz.observe(lassochanges, names="lassoedPointIDs")
viz.observe(selectedanchorchanges, names="selectedAnchorID")
viz.observe(selecteddatachanges, names="selectedDataPointID")

anchorchanges(None)
datachanges(None)
lassochanges(None)

In [7]:
logging.warning("help")

In [8]:
viz._anchor_drag_callbacks.callbacks[0]("test")

In [9]:
viz

AnchorViz(anchors=[{'id': '0', 'name': 'test anchor', 'color': 'red', 'theta': -0.5}, {'id': '1', 'name': 'tes…

In [10]:
HBox([viz, VBox([anchor_display, data_display]), VBox([lassoed_data_display, selected_anchor_display, selected_point_display, what_display])])

Try adding two anchors above.

In [18]:
import random
def gen_big_data(n=100):
    points = []
    for i in range(n):
        point = dict(id=str(i), color=("blue" if i%2 == 0 else "orange"), labeled=(i%5 == 0), weights=dict())
        for key in ["0", "1", "2", "3"]:
            val = random.random()
            if key == "0" and i%2 == 0:
                val += .5
                val *= 4
            point["weights"][key] = val
        points.append(point)
    return points

In [19]:
viz.data = gen_big_data()

In [20]:
viz.modify_anchor(0, "name", "Blue feature")
viz.modify_anchor(1, "name", "Orange feature 1")
viz.modify_anchor(2, "name", "Orange feature 2")
viz.modify_anchor(3, "name", "Orange feature 3")

In [16]:
alt_data = [
    {"id": "0", "color": "blue", "weights": { "0": 0.5, "1": 0.6}, "labeled": True},
    {"id": "1", "color": "blue", "weights": { "0": 0.0, "1": 0.1}, "labeled": True},
    {"id": "2", "color": "blue", "weights": { "0": 0.4, "1": 1.1}, "labeled": True},
    {"id": "3", "color": "blue", "weights": { "0": 0.0, "1": 1.0}, "labeled": True},
    {"id": "4", "color": "blue", "weights": { "0": 0.0, "1": 1.5}},
    {"id": "5", "color": "blue", "weights": { "0": 0.7, "1": 1.0}, "labeled": True},
    {"id": "6", "color": "blue", "weights": { "0": 0.5, "1": 0.5}},
    {"id": "7", "color": "blue", "weights": { "0": 0.4, "1": 0.7}},
]
viz.data = alt_data

In [17]:
viz.modify_point(3, "color", "green")
viz.modify_point(4, "color", "green")

In [ ]:
viz.add_point({"color": "yellow", "weights": { "0": 0.6, "1": 0.2 }})

In [ ]:
viz.lassoedPointIDs = [1, 2]

In [ ]:
viz.selectedAnchorID = "1"

In [ ]:
viz.selectedDataPointID = "0"

In [ ]:
viz.anchors = [
    {"id": "0", "name": "things", "theta": 1.5},
    {"id": "1", "name": "things2", "theta": 2.5}
]

Add anchor via python

In [ ]:
# type(viz.anchors), type(viz.anchors[0])

In [ ]:
# viz.anchors = viz.anchors + [{"id": 5, "name": "things", "theta": 1.5}]

Modify anchor via python

In [ ]:
viz.modify_anchor(1, "color", "purple")

In [ ]:
anchor_to_change = viz.anchors[1]
anchor_to_change["color"] = "purple"
viz.anchors.pop(1)
viz.anchors = viz.anchors + [anchor_to_change]


In [ ]:
viz.anchors[1]

In [ ]:
viz.anchors[1] = dict(id=1, name="whaaaa", theta="0.5", color="orange")

In [ ]:
type(viz.anchors)

In [ ]:
viz.anchors[1]["color"] = "orange"

In [ ]:
viz.anchors[1]["color"] = "orange"
viz.notify_change({"name": "anchors", "type":"change"})

In [ ]:
anchors = viz.anchors
anchors[1]["color"] = "green"
viz.anchors = anchors

Delete anchor via python

In [21]:
viz.anchors = [viz.anchors[0]]

In [ ]:
viz.anchors